In [1]:
import matplotlib.pyplot as plt
plt.rc('font', size=12)
import pandas as pd
import seaborn as sns
import warnings 

import Coverage as cov

warnings.filterwarnings('ignore')
%matplotlib inline

%load_ext autoreload
%autoreload 2

## Params

In [2]:
#get list of client rules sorted by count
popSize = 10
clientFilename = "../Data/ICU/Best/"
# clientFilename = "../Data/ICU/Test/"
# clientFilename = "../Data/Sepsis/Best/"

ldpFilename = "../Results/ICU_Base_TEST.csv"
# ldpFilename = "../Results/ICU_Ruleset_MCTS_TEST.csv"
# ldpFilename = "../Results/ICU_TEST_2.csv"
# ldpFilename = "../Results/Sepsis_Ruleset_MCTS_Test.csv"

graphName = '../Results/Graphs/test' #Name of count coverage graphs
popThresh = 0.01 #Percentage match count


## Load Client Rules

In [3]:
# Load Client Rule Trees and Text Rule Structures, and get dataframe of rules and their counts
clientTrees, clientRules, clientDF = cov.loadClientRules(popSize, clientFilename)
clientDF

File not found for Client 8


,Rule,Rule Count,Percent of Population
53,"((MET >= 0.000) U[0,0] (death = 0.000))",17,1.0
56,"F[0,0]((BLOOD_UREA_NITROGEN <= 0.000 & CREATIN...",13,1.0
57,"G[0,0]((n_evts <= 0.000 -> LOS >= 0.000))",12,1.0
47,"G[0,0]((hr >= 0.000 & Pulse >= 0.000))",3,0.3
172,"((af >= 0.000) U[0,0] (direct >= 0.000))",3,0.3
...,...,...,...
188,"F[0,0]((HEMOGLOBIN <= 0.000 & O2_Flow <= 0.000))",1,0.1
187,"F[0,0]((dfa >= 0.000 & Mort >= 0.000))",1,0.1
186,"((O2_Flow <= 0.000) U[0,0] (PLATELET_COUNT >= ...",1,0.1
185,"((MET >= 0.000) U[0,0] (PARTIAL_THROMBOPLASTIN...",1,0.1


In [4]:
clientDF.to_csv("Client Rules.csv")

## Load LDP Ruleset

In [5]:
ldpDF, ldpTrees, ldpRules = cov.loadLDPRuleset(ldpFilename)
ldpDF

FileNotFoundError: [Errno 2] No such file or directory: '../Results/ICU_Base_TEST.csv'

## Complexity Analysis

In [ ]:
k = 4 #k-ary tree
h = 17 #height
k**h

## Get Table of Structure Counts - Count of rules above threshold

In [ ]:
covDF, countDF, clientTrees = cov.getCoverageTable(popThresh, ldpDF, ldpTrees, clientDF)
countDF.to_csv(ldpFilename + "_CovCountDF.csv")
covDF

In [ ]:
countDF

## Get number of unique structure types

In [ ]:
structDF = cov.countUniqueStructuresNoVars(clientTrees, ldpTrees)
structDF

## Compare LDP Rules Found vs the Percent Client Counts 

In [ ]:
#make list of rule counts x population percents

lst = []
for idx, row in clientDF.iterrows():
    lst.extend([row['Percent of Population'] for i in range(row['Rule Count'])])
    
n_bins = 10

plt.figure(figsize=(12, 7))
plt.hist([lst, countDF['Client Count'].values], n_bins, density=True, histtype='bar',label=['Client', 'LDP'])
plt.xlabel("Percentage of Population")
plt.ylabel("Number of Rules")
plt.yscale('log')
plt.legend()


In [ ]:
from SignalTemporalLogic.STLFactory import STLFactory
from Client import Client
from Server import Server

In [ ]:
# From client rules, first make client trees
clientRules = ['F[0,0]((death <= 0.000 & y >= 0.000))']
clientTrees = []
stlFac = STLFactory()

for c in clientRules:
    c = stlFac.constructFormulaTree(c + "\n")
    clientTrees.append(c)
    
clientTrees

In [ ]:
ldpRules = ['F[0,0](death > 0.000 & y = 0.000)']
ldpTrees = []
stlFac = STLFactory()

for c in ldpRules:
    c = stlFac.constructFormulaTree(c + "\n")
    ldpTrees.append(c)
    
ldpTrees

In [ ]:
def queryStructuralFullMatch(template, clientTrees):

    client = Client(clientNum=1, epsilon='inf', ruleSet=clientTrees)
    server = Server(clientList=[], varDict={}, params=None)

#     print("Temp vars", varList)
#     print("\ntemp", template.toString())
    ldpNodes = server.getTemplateNodes(template)
    ldpVars = template.getAllVars()

#     print("templt nodes", ldpNodes)

    r = client.queryStructuralRuleMatchReturn(ldpNodes, ldpVars)

    return r

In [ ]:
found = queryStructuralFullMatch(ldpTrees[0], clientTrees)
print(found)

In [ ]:
def nodeListMatch(tempList, cList):
    relops = ['GT', 'GE', 'LT', 'LE', "EQ"]  # , 'NEQ']

    i = 0
    while i < len(tempList):
        # get current branch of nodes
        if tempList[i] in cList:
            if 'Parameter' not in tempList[i]:  # found a var match
                #                 idx = cList.index(tempList[i])  # get idx of element of cList
                #                 cList = cList[idx + 1:]
                cList.remove(tempList[i])

        elif any(item in tempList[i] for item in relops):
            # Try all match options
            if 'GT' in tempList[i]:
                test = [x if x != "GT" else "GE" for x in tempList[i]]

                if test in cList:
                    cList.remove(test)
                else:
                    return False
            elif 'GE' in tempList[i]:
                test1 = [x if x != "GE" else "GT" for x in tempList[i]]
                test2 = [x if x != "GE" else "EQ" for x in tempList[i]]

                try:
                    idx1 = cList.index(test1)
                except:
                    idx1 = 999999999
                try:
                    idx2 = cList.index(test2)
                except:
                    idx2 = 999999999

                if test1 in cList and (idx1 < idx2):
                    cList.remove(test1)
                elif test2 in cList and (idx2 < idx1):
                    cList.remove(test2)
                else:
                    return False
            elif 'LT' in tempList[i]:
                test = [x if x != "LT" else "LE" for x in tempList[i]]
                if test in cList:
                    cList.remove(test)
                else:
                    return False
            elif 'LE' in tempList[i]:
                test1 = [x if x != "LE" else "LT" for x in tempList[i]]
                test2 = [x if x != "LE" else "EQ" for x in tempList[i]]
                try:
                    idx1 = cList.index(test1)
                except:
                    idx1 = 999999999
                try:
                    idx2 = cList.index(test2)
                except:
                    idx2 = 999999999

                if test1 in cList and (idx1 < idx2):
                    cList.remove(test1)
                elif test2 in cList and (idx2 < idx1):
                    cList.remove(test2)
                else:
                    return False
            elif 'EQ' in tempList[i]:
                test1 = [x if x != "EQ" else "GE" for x in tempList[i]]
                test2 = [x if x != "EQ" else "LE" for x in tempList[i]]

                try:
                    idx1 = cList.index(test1)
                except:
                    idx1 = 999999999
                try:
                    idx2 = cList.index(test2)
                except:
                    idx2 = 999999999

                if test1 in cList and (idx1 < idx2):
                    cList.remove(test1)
                elif test2 in cList and (idx2 < idx1):
                    cList.remove(test2)
                else:
                    return False

        else:
            return False

        i = i + 1

    return True

In [ ]:
tempList = [['eval', 0], ['statementList', 1], ['statement', 2], ['boolExpr', 3], ['stlTerm', 4], ['BooleanAtomic', 'F', 5], ['boolExpr', 6], ['stlTerm', 7], ['BooleanAtomic', 8], ['boolExpr', 9], ['AND', 'stlTerm', 'stlTerm', 10], ['BooleanAtomic', 11], ['BooleanAtomic', 11], ['GE', 'ICU_Pt_Days', 'Parameter', 12], ['LE', 'n_edrk', 'Parameter', 12]]
cList = [['eval', 0], ['statementList', 1], ['statement', 2], ['boolExpr', 3], ['stlTerm', 4], ['BooleanAtomic', 'F', 5], ['boolExpr', 6], ['stlTerm', 7], ['BooleanAtomic', 8], ['boolExpr', 9], ['AND', 'stlTerm', 'stlTerm', 10], ['BooleanAtomic', 11], ['BooleanAtomic', 11], ['EQ', 'n_edrk', 'Parameter', 12], ['LT', 'ICU_Pt_Days', 'Parameter', 12]]
nodeListMatch(tempList, cList)


In [ ]:
tempList = [['eval', 0], ['statementList', 1], ['statement', 2], ['boolExpr', 3], ['stlTerm', 4], ['BooleanAtomic', 'F', 5], ['boolExpr', 6], ['AND', 'stlTerm', 'stlTerm', 7], ['BooleanAtomic', 8], ['BooleanAtomic', 8], ['EQ', 'death', 'Parameter', 9], ['GT', 'y', 'Parameter', 9]]
cList = [['eval', 0], ['statementList', 1], ['statement', 2], ['boolExpr', 3], ['stlTerm', 4], ['BooleanAtomic', 'F', 5], ['boolExpr', 6], ['AND', 'stlTerm', 'stlTerm', 7], ['BooleanAtomic', 8], ['BooleanAtomic', 8], ['LE', 'death', 'Parameter', 9], ['GE', 'y', 'Parameter', 9]]

nodeListMatch(tempList, cList)


In [ ]:
stlFac = STLFactory()
temp = stlFac.constructFormulaTree("F[?,?](y = ? & death > ?)\n")

server = Server(clientList=[], varDict={}, params=None)
server.getTemplateNodes(temp)

In [ ]:
stlFac.constructFormulaTree(temp.toStringWithParams())

In [ ]:
getTemplateNodes(temp)